<a href="https://colab.research.google.com/github/Curly-And-Straight/Machine-Learning-Clustering/blob/main/MainImageClassifierCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras import applications

from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
from keras.utils import to_categorical
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.applications import EfficientNetB4

import keras

In [2]:
batch_size = 32
epoch = 15
train_path = 'drive/MyDrive/dataset/train/images'
valid_path = 'drive/MyDrive/dataset/test/images'

In [4]:
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory(train_path, class_mode="categorical", shuffle=False, batch_size=batch_size, target_size=(330, 330))
# load val data
val = imagegen.flow_from_directory(valid_path, class_mode="categorical", shuffle=False, batch_size=batch_size, target_size=(330, 330))

Found 532 images belonging to 19 classes.
Found 380 images belonging to 19 classes.


In [5]:
# pretrained_model = VGG16(include_top=False, weights='imagenet')

In [6]:
# pretrained_model = applications.resnet50.ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

In [7]:
# pretrained_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape= (224,224,3))

In [8]:
# pretrained_model = Xception(weights='imagenet', include_top=False, input_shape= (224,224,3))

In [9]:
# pretrained_model = InceptionV3(weights='imagenet', include_top=False, input_shape= (224,224,3))

In [10]:
# pretrained_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape= (224,224,3))

In [11]:
# pretrained_model = NASNetLarge(weights='imagenet', include_top=False, input_shape= (331,331,3))

In [12]:
pretrained_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape= (330,330,3))

In [13]:
pretrained_model.trainable = False

In [14]:
pretrained_model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 330, 330, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 330, 330, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 330, 330, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 331, 331, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [15]:
# extract train and val features
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

In [16]:
# OHE target column
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [17]:
model = Sequential()

model.add(Flatten())

model.add(Dense(512, activation='selu'))
model.add(Dropout(0.6))
model.add(BatchNormalization())

model.add(Dense(256, activation='selu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(128, activation='selu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(64, activation='selu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(19, activation='softmax'))

In [18]:
# compile the model
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 216832)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               111018496 
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1

In [20]:
# train model using features generated from EfficienNetB4 model
history = model.fit(vgg_features_train, train_target, epochs=epoch, batch_size=128, validation_data=(vgg_features_val, val_target))

Epoch 1/15
5/5 [==============================] - 8s 1s/step - loss: 3.5092 - accuracy: 0.0526 - val_loss: 6.1292 - val_accuracy: 0.1711
Epoch 2/15
5/5 [==============================] - 5s 1s/step - loss: 3.1991 - accuracy: 0.1071 - val_loss: 3.6019 - val_accuracy: 0.2263
Epoch 3/15
5/5 [==============================] - 5s 1s/step - loss: 2.9848 - accuracy: 0.1304 - val_loss: 2.7506 - val_accuracy: 0.3368
Epoch 4/15
5/5 [==============================] - 5s 1s/step - loss: 2.8714 - accuracy: 0.1644 - val_loss: 2.4322 - val_accuracy: 0.3632
Epoch 5/15
5/5 [==============================] - 5s 1s/step - loss: 2.7501 - accuracy: 0.1971 - val_loss: 2.2270 - val_accuracy: 0.4237
Epoch 6/15
5/5 [==============================] - 5s 1s/step - loss: 2.6316 - accuracy: 0.2145 - val_loss: 2.0762 - val_accuracy: 0.4842
Epoch 7/15
5/5 [==============================] - 5s 1s/step - loss: 2.6364 - accuracy: 0.2049 - val_loss: 1.9578 - val_accuracy: 0.5184
Epoch 8/15
5/5 [=========================

In [21]:
pretrained_model.trainable = True

In [22]:
print("Number of layers in the base model: ", len(pretrained_model.layers))

Number of layers in the base model:  474


In [23]:
fine_tune_at = 430

for layer in pretrained_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [24]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [26]:
fine_tune_epochs = epoch

total_epochs =  epoch + fine_tune_epochs

history2 = model.fit(vgg_features_train, train_target, epochs=total_epochs, initial_epoch=history.epoch[-1], batch_size=128, validation_data=(vgg_features_val, val_target))

Epoch 15/30
5/5 [==============================] - 7s 1s/step - loss: 1.7427 - accuracy: 0.4779 - val_loss: 1.4155 - val_accuracy: 0.6342
Epoch 16/30
5/5 [==============================] - 5s 1s/step - loss: 1.7626 - accuracy: 0.4907 - val_loss: 1.3604 - val_accuracy: 0.6500
Epoch 17/30
5/5 [==============================] - 5s 1s/step - loss: 1.5991 - accuracy: 0.5301 - val_loss: 1.3933 - val_accuracy: 0.6368
Epoch 18/30
5/5 [==============================] - 5s 1s/step - loss: 1.5440 - accuracy: 0.5417 - val_loss: 1.3583 - val_accuracy: 0.6500
Epoch 19/30
5/5 [==============================] - 5s 1s/step - loss: 1.3818 - accuracy: 0.5936 - val_loss: 1.3173 - val_accuracy: 0.6500
Epoch 20/30
5/5 [==============================] - 5s 1s/step - loss: 1.3542 - accuracy: 0.6070 - val_loss: 1.2950 - val_accuracy: 0.6579
Epoch 21/30
5/5 [==============================] - 5s 1s/step - loss: 1.2752 - accuracy: 0.6338 - val_loss: 1.2491 - val_accuracy: 0.6737
Epoch 22/30
5/5 [=================

In [28]:
# model.save("drive/MyDrive/cnn_model.h5")